In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,43287
2,Huelva,Confirmados PDIA 14 días,2131
3,Huelva,Tasa PDIA 14 días,"415,26199894771713"
4,Huelva,Confirmados PDIA 7 días,911
5,Huelva,Tasa PDIA 7 dias,"177,52401738215406"
6,Huelva,Total Confirmados,43486
7,Huelva,Curados,36977
8,Huelva,Fallecidos,399


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  43287.0


/tmp/ipykernel_3275/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12089.0


/tmp/ipykernel_3275/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3275/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3275/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3275/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 545 personas en los últimos 7 días 

Un positivo PCR cada 230 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,43287.0,911.0,2131.0,513170.0,177.524017,415.261999,79.0
Huelva-Costa,25524.0,607.0,1385.0,289548.0,209.637090,478.331745,43.0
Condado-Campiña,13534.0,220.0,577.0,156231.0,140.817123,369.324910,27.0
Huelva (capital),12089.0,264.0,626.0,143837.0,183.541092,435.214861,19.0
Sierra de Huelva-Andévalo Central,3828.0,74.0,153.0,67391.0,109.806948,227.033283,8.0
Almonte,2187.0,29.0,67.0,24507.0,118.333537,273.391276,8.0
San Juan del Puerto,727.0,18.0,34.0,9411.0,191.265540,361.279354,7.0
Lepe,2865.0,59.0,167.0,27880.0,211.621234,598.995696,6.0
Valverde del Camino,814.0,16.0,23.0,12750.0,125.490196,180.392157,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Linares de la Sierra,9.0,4.0,5.0,262.0,1526.717557,1908.396947,0.0
Alájar,31.0,7.0,12.0,759.0,922.266140,1581.027668,1.0
Cañaveral de León,21.0,2.0,6.0,396.0,505.050505,1515.151515,0.0
Corteconcepción,45.0,2.0,6.0,536.0,373.134328,1119.402985,0.0
Ayamonte,1920.0,83.0,160.0,21104.0,393.290371,758.150114,1.0
Isla Cristina,2942.0,76.0,160.0,21393.0,355.256392,747.908194,6.0
Campofrío,20.0,2.0,5.0,713.0,280.504909,701.262272,0.0
Villablanca,265.0,9.0,20.0,2885.0,311.958406,693.240901,2.0
Rosal de la Frontera,56.0,10.0,11.0,1697.0,589.275192,648.202711,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,136.0,1.0,6.0,2118.0,47.214353,283.286119,0.0,0.166667
Escacena del Campo,209.0,2.0,11.0,2287.0,87.450809,480.979449,0.0,0.181818
Cortegana,283.0,3.0,16.0,4602.0,65.189048,347.674924,0.0,0.187500
Trigueros,523.0,9.0,48.0,7862.0,114.474688,610.531671,0.0,0.187500
Paterna del Campo,191.0,3.0,16.0,3457.0,86.780445,462.829043,1.0,0.187500
Villarrasa,216.0,2.0,10.0,2211.0,90.456807,452.284034,0.0,0.200000
Bonares,442.0,6.0,28.0,6060.0,99.009901,462.046205,1.0,0.214286
Lucena del Puerto,345.0,2.0,9.0,3261.0,61.330880,275.988960,2.0,0.222222
Aracena,646.0,6.0,22.0,8255.0,72.683222,266.505148,1.0,0.272727
